In [1]:
#Connect to wandb
#TO DO : how to save models on the weight and bias platform
import wandb
wandb.login()
wandb.init(project="xgb", 
           entity="benchmark-nlp",
           name='topic datasets') #CHANGE

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jtonglet (benchmark-nlp). Use `wandb login --relogin` to force relogin


In [2]:
import os
os.chdir('../../..')

In [3]:
#Load packages
import warnings
import io
import numpy as np
import pandas as pd
from codecarbon import EmissionsTracker
import yaml
from util.dataloader import DataLoader
from preprocessing.preprocessor import Preprocessor
from util.datasplitter import data_splitter
from preprocessing.fasttext_embeddings import FastTextEmbeddings
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, average_precision_score
warnings.filterwarnings("ignore")

In [4]:
#Set constant values
SEED=42 
OPT_ITER=20

## Load data

In [5]:
dl = DataLoader(['topic'])
data = dl.load()


tweet_preprocessor = Preprocessor(is_tweet=True)
preprocessor = Preprocessor()

#We are not interested in the test sets for hyperparameter optimization
train_twentynews, val_twentynews, _ = data_splitter(data['twentynews'],
                                 preprocessor, 
                                 create_val_set=True,   #No validation set is provided
                                 seed=SEED)
train_agnews, val_agnews, _ = data_splitter(data['agnews'],
                                 preprocessor, 
                                 create_val_set=True,   #No validation set is provided
                                 seed=SEED)
train_yahoo, val_yahoo, _ = data_splitter(data['yahoo'],
                                 preprocessor, 
                                 create_val_set=True,   #No validation set is provided
                                 seed=SEED)

120000 rows preprocessed in 66.11223077774048 seconds
7600 rows preprocessed in 3.4647884368896484 seconds


In [6]:
%%time
#fasttext 
fasttext = FastTextEmbeddings()
fasttext.load_model('fasttext/cc.en.300.bin')

CPU times: total: 30.2 s
Wall time: 48.2 s

In [7]:
embedded_train_twentynews = fasttext.generate_sentence_embeddings(train_twentynews['text'])
embedded_val_twentynews = fasttext.generate_sentence_embeddings(val_twentynews['text'])
embedded_train_twentynews['label'] = train_twentynews['label'].to_list()
embedded_val_twentynews['label'] = val_twentynews['label'].to_list()

Starting to generate sentence embeddings


100%|██████████████████████████████████████████████████████████████████████████████| 9051/9051 [01:59<00:00, 75.62it/s]


Starting to generate sentence embeddings


100%|█████████████████████████████████████████████████████████████████████████████| 2263/2263 [00:17<00:00, 127.40it/s]


In [7]:
embedded_train_agnews = fasttext.generate_sentence_embeddings(train_agnews['text'])
embedded_val_agnews = fasttext.generate_sentence_embeddings(val_agnews['text'])
embedded_train_agnews['label'] = train_agnews['label'].to_list()
embedded_val_agnews['label'] = val_agnews['label'].to_list()

Starting to generate sentence embeddings


100%|███████████████████████████████████████████████████████████████████████████| 96000/96000 [04:35<00:00, 348.01it/s]


Starting to generate sentence embeddings


100%|███████████████████████████████████████████████████████████████████████████| 24000/24000 [01:14<00:00, 320.24it/s]


In [8]:
embedded_train_yahoo = fasttext.generate_sentence_embeddings(train_yahoo['text'])
embedded_val_yahoo = fasttext.generate_sentence_embeddings(val_yahoo['text'])
embedded_train_yahoo['label'] = train_yahoo['label'].to_list()
embedded_val_yahoo['label'] = val_yahoo['label'].to_list()

starting to generate sentence embeddings


100%|█████████████████████████████████████████████████████████████████████| 1120000/1120000 [1:10:51<00:00, 263.45it/s]


starting to generate sentence embeddings


100%|█████████████████████████████████████████████████████████████████████████| 280000/280000 [19:35<00:00, 238.10it/s]


## Hyperopt

In [8]:
import yaml
#Load the template yaml sweep config file for logistic regression
#If the value range for an hyperparameter needs to be changed, better to do it in the .yaml file than in a notebook
with open("config/xgb_sweep.yaml", 'r') as stream:
    sweep_config = yaml.safe_load(stream)

In [9]:
#The config is displayed as a nested dictionary
sweep_config

{'method': 'random',
 'entity': 'benchmark-nlp',
 'project': 'hyperopt',
 'metric': {'name': 'loss', 'goal': 'minimize'},
 'parameters': {'gamma': {'min': 0, 'max': 1, 'distribution': 'uniform'},
  'n_estimators': {'min': 10, 'max': 150, 'distribution': 'int_uniform'},
  'learning_rate': {'min': 0.001, 'max': 0.1, 'distribution': 'uniform'},
  'max_depth': {'min': 0, 'max': 10, 'distribution': 'int_uniform'},
  'random_state': {'value': 42}}}

#### twenty news   Tf-Idf

In [10]:
#Don't forget to name the sweep instance  
name = 'xgb_tfidf_twentynews' #change here
sweep_config['name'] =  name

#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="xgb")

def train_tfidf(config = None,
          train=train_twentynews, #Change here
          val=val_twentynews): #change here
    '''
    Generic WandB function to conduct hyperparameter optimization with tf-idf vectorizer
    '''
    # Initialize a new wandb run
    with wandb.init(config=config,group=name):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        vec = TfidfVectorizer()
        clf = XGBClassifier(gamma=config.gamma,
                                 n_estimators=config.n_estimators,
                                 learning_rate=config.learning_rate,
                                 max_depth=config.max_depth,
                                 random_state=config.random_state) #set the hyperparams here
        
        #Create the pipeline
        pipe = Pipeline([('vectorizer',vec),('clf',clf)])
        #Fit the pipeline
        pipe.fit(train['text'],train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val['text'])
        pred_prob_val = pipe.predict_proba(val['text'])[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
        if train['label'].nunique() <=2:
            aucpc =  average_precision_score(val['label'],pred_prob_val)
            auc = roc_auc_score(val['label'],pred_prob_val)
        else:
            aucpc = '-'
            auc = '-'
        #Log metrics on WandB
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })

#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_tfidf,count=OPT_ITER) #Count : number of iterations
tracker.stop()

Create sweep with ID: 4fbrlrzg
Sweep URL: https://wandb.ai/benchmark-nlp/xgb/sweeps/4fbrlrzg


[codecarbon WARNING @ 12:16:38] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 12:16:41] We saw that you have a Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz but we don't know it. Please contact us.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: k4dgcf3w with config:
wandb: 	gamma: 0.03752195732255115
wandb: 	learning_rate: 0.054755228213609314
wandb: 	max_depth: 9
wandb: 	n_estimators: 93
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.63544
f1 macro,0.63184


wandb: Agent Starting Run: f9nacfrv with config:
wandb: 	gamma: 0.1009587978946167
wandb: 	learning_rate: 0.07181920317070158
wandb: 	max_depth: 7
wandb: 	n_estimators: 77
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.62483
f1 macro,0.62238


...

wandb: Agent Starting Run: cyk1ored with config:
wandb: 	gamma: 0.3312356548350702
wandb: 	learning_rate: 0.04700386955694736
wandb: 	max_depth: 0
wandb: 	n_estimators: 122
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.04507
f1 macro,0.00431


wandb: Agent Starting Run: n5nh11lp with config:
wandb: 	gamma: 0.8104623257014634
wandb: 	learning_rate: 0.009157205322550096
wandb: 	max_depth: 3
wandb: 	n_estimators: 87
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.5232
f1 macro,0.55099


0.01084978951487711

In [11]:
#Don't forget to name the sweep instance   
name = 'xgb_ft_twentynews' #change here
sweep_config['name'] = name
#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="xgb")

def train_fasttext(config = None,
          train=embedded_train_twentynews, #Change here
          val=embedded_val_twentynews): #change here
    # Initialize a new wandb run
    with wandb.init(config=config, group=name):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        clf = XGBClassifier(gamma=config.gamma,
                                 n_estimators=config.n_estimators,
                                 learning_rate=config.learning_rate,
                                 max_depth=config.max_depth,
                                 random_state=config.random_state) #set the hyperparams here
        pipe = Pipeline([('clf',clf)])
        pipe.fit(train.fillna(0).drop(['label'],axis=1),train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val.fillna(0).drop(['label'],axis=1))
        pred_prob_val = pipe.predict_proba(val.fillna(0).drop(['label'],axis=1))[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
        if train['label'].nunique() <=2:
            aucpc =  average_precision_score(val['label'],pred_prob_val)
            auc = roc_auc_score(val['label'],pred_prob_val)
            #Log predictions on WandB
        else:
            aucpc = '-'
            auc = '-'
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })


#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_fasttext,count=OPT_ITER)
tracker.stop()

[codecarbon WARNING @ 13:28:58] No CPU tracking mode found. Falling back on CPU constant mode.


Create sweep with ID: dhu4wzbf
Sweep URL: https://wandb.ai/benchmark-nlp/xgb/sweeps/dhu4wzbf


[codecarbon WARNING @ 13:29:01] We saw that you have a Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz but we don't know it. Please contact us.
wandb: Agent Starting Run: 4348ggqo with config:
wandb: 	gamma: 0.2656303509439727
wandb: 	learning_rate: 0.0018291344176754349
wandb: 	max_depth: 9
wandb: 	n_estimators: 45
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.43129
f1 macro,0.42427


wandb: Agent Starting Run: helb9qs5 with config:
wandb: 	gamma: 0.576933686154143
wandb: 	learning_rate: 0.0265968217591904
wandb: 	max_depth: 10
wandb: 	n_estimators: 14
wandb: 	random_state: 42


...

wandb: Agent Starting Run: enkv12f6 with config:
wandb: 	gamma: 0.4065655892429949
wandb: 	learning_rate: 0.02038825587498175
wandb: 	max_depth: 3
wandb: 	n_estimators: 25
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.44764
f1 macro,0.4297


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 3xq75g7o with config:
wandb: 	gamma: 0.2011395388105972
wandb: 	learning_rate: 0.010623386023852993
wandb: 	max_depth: 4
wandb: 	n_estimators: 56
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.49006
f1 macro,0.47531


0.016024379952925192

In [12]:
del train_twentynews
del val_twentynews
del embedded_train_twentynews
del embedded_val_twentynews

#### agnews tfidf

In [8]:
#Don't forget to name the sweep instance  
name = 'xgb_tfidf_agnews' #change here
sweep_config['name'] =  name

#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="xgb")

def train_tfidf(config = None,
          train=train_agnews, #Change here
          val=val_agnews): #change here
    '''
    Generic WandB function to conduct hyperparameter optimization with tf-idf vectorizer
    '''
    # Initialize a new wandb run
    with wandb.init(config=config,group=name):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        vec = TfidfVectorizer()
        clf = XGBClassifier(gamma=config.gamma,
                                 n_estimators=config.n_estimators,
                                 learning_rate=config.learning_rate,
                                 max_depth=config.max_depth,
                                 random_state=config.random_state) #set the hyperparams here
        
        #Create the pipeline
        pipe = Pipeline([('vectorizer',vec),('clf',clf)])
        #Fit the pipeline
        pipe.fit(train['text'],train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val['text'])
        pred_prob_val = pipe.predict_proba(val['text'])[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
        if train['label'].nunique() <=2:
            aucpc =  average_precision_score(val['label'],pred_prob_val)
            auc = roc_auc_score(val['label'],pred_prob_val)
        else:
            aucpc = '-'
            auc = '-'
        #Log metrics on WandB
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })

#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_tfidf,count=OPT_ITER)
tracker.stop()

[codecarbon WARNING @ 12:50:25] No CPU tracking mode found. Falling back on CPU constant mode.


Create sweep with ID: gkpw9mj1
Sweep URL: https://wandb.ai/benchmark-nlp/xgb/sweeps/gkpw9mj1


[codecarbon WARNING @ 12:50:28] We saw that you have a Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz but we don't know it. Please contact us.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: 8ynsxcw1 with config:
wandb: 	gamma: 0.5023710420193381
wandb: 	learning_rate: 0.01223540129501244
wandb: 	max_depth: 8
wandb: 	n_estimators: 92
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.72671
f1 macro,0.73546


wandb: Agent Starting Run: dfmuiyaq with config:
wandb: 	gamma: 0.6309087925819006
wandb: 	learning_rate: 0.09972042883246456
wandb: 	max_depth: 0
wandb: 	n_estimators: 21
wandb: 	random_state: 42


...

wandb: Agent Starting Run: 41h6l5yj with config:
wandb: 	gamma: 0.13302856345369685
wandb: 	learning_rate: 0.05247245014944386
wandb: 	max_depth: 1
wandb: 	n_estimators: 122
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.68817
f1 macro,0.69822


wandb: Agent Starting Run: z1uc9akl with config:
wandb: 	gamma: 0.791633212142687
wandb: 	learning_rate: 0.026603168950532944
wandb: 	max_depth: 0
wandb: 	n_estimators: 108
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.24629
f1 macro,0.09881


0.009679389894775407

In [ ]:
#Don't forget to name the sweep instance   
name = 'xgb_ft_agnews' #CHANGE HERE
sweep_config['name'] = name
#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="xgb")

def train_fasttext(config = None,
          train=embedded_train_agnews, #CHANGE HERE
          val=embedded_val_agnews): #CHANGE HERE
    # Initialize a new wandb run
    with wandb.init(config=config, group=name):
        config = wandb.config
        clf = XGBClassifier(gamma=config.gamma,
                                 n_estimators=config.n_estimators,
                                 learning_rate=config.learning_rate,
                                 max_depth=config.max_depth,
                                 random_state=config.random_state) #set the hyperparams here
        pipe = Pipeline([('clf',clf)])
        pipe.fit(train.fillna(0).drop(['label'],axis=1),train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val.fillna(0).drop(['label'],axis=1))
        pred_prob_val = pipe.predict_proba(val.fillna(0).drop(['label'],axis=1))[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
        if train['label'].nunique() <=2:
            aucpc =  average_precision_score(val['label'],pred_prob_val)
            auc = roc_auc_score(val['label'],pred_prob_val)
            #Log predictions on WandB
        else:
            aucpc = '-'
            auc = '-'
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })


#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_fasttext,count=OPT_ITER)
tracker.stop()  #13 iterations remaining

[codecarbon WARNING @ 15:28:01] No CPU tracking mode found. Falling back on CPU constant mode.


Create sweep with ID: rzu2pnun
Sweep URL: https://wandb.ai/benchmark-nlp/xgb/sweeps/rzu2pnun


[codecarbon WARNING @ 15:28:04] We saw that you have a Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz but we don't know it. Please contact us.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: xtruiii0 with config:
wandb: 	gamma: 0.746700151413228
wandb: 	learning_rate: 0.045952256010118264
wandb: 	max_depth: 8
wandb: 	n_estimators: 77
wandb: 	random_state: 42


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.87242
f1 macro,0.87187


wandb: Agent Starting Run: yn2ey9vy with config:
wandb: 	gamma: 0.70142947397209
wandb: 	learning_rate: 0.07035052028549085
wandb: 	max_depth: 8
wandb: 	n_estimators: 30
wandb: 	random_state: 42


In [ ]:
del train_agnews
del val_agnews
del embedded_train_agnews
del embedded_val_agnews

#### yahoo

In [15]:
#Don't forget to name the sweep instance  
name = 'xgb_tfidf_yahoo' #change here
sweep_config['name'] =  name

#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="xgb")

def train_tfidf(config = None,
          train=train_yahoo, #Change here
          val=val_yahoo): #change here
    '''
    Generic WandB function to conduct hyperparameter optimization with tf-idf vectorizer
    '''
    # Initialize a new wandb run
    with wandb.init(config=config,group=name):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        vec = TfidfVectorizer()
        clf = XGBClassifier(gamma=config.gamma,
                                 n_estimators=config.n_estimators,
                                 learning_rate=config.learning_rate,
                                 max_depth=config.max_depth,
                                 random_state=config.random_state) #set the hyperparams here
        
        #Create the pipeline
        pipe = Pipeline([('vectorizer',vec),('clf',clf)])
        #Fit the pipeline
        pipe.fit(train['text'],train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val['text'])
        pred_prob_val = pipe.predict_proba(val['text'])[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
        if train['label'].nunique() <=2:
            aucpc =  average_precision_score(val['label'],pred_prob_val)
            auc = roc_auc_score(val['label'],pred_prob_val)
        else:
            aucpc = '-'
            auc = '-'
        #Log metrics on WandB
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })

#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_tfidf, count=OPT_ITER)
tracker.stop()

[codecarbon WARNING @ 13:24:16] No CPU tracking mode found. Falling back on CPU constant mode.


Create sweep with ID: x86sj11j
Sweep URL: https://wandb.ai/benchmark-nlp/hyperopt/sweeps/x86sj11j


[codecarbon WARNING @ 13:24:19] We saw that you have a Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz but we don't know it. Please contact us.
wandb: Agent Starting Run: af0cnjkn with config:
wandb: 	C: 5.905437512178784
wandb: 	penalty: l2
wandb: 	random_state: 42
wandb: 	solver: lbfgs


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.44918
f1 macro,0.50039


wandb: Agent Starting Run: h7uylwu8 with config:
wandb: 	C: 7.619241650157256
wandb: 	penalty: l2
wandb: 	random_state: 42
wandb: 	solver: lbfgs


...

wandb: Agent Starting Run: cwdgf5d1 with config:
wandb: 	C: 7.717716747047102
wandb: 	penalty: l2
wandb: 	random_state: 42
wandb: 	solver: lbfgs


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.44783
f1 macro,0.49873


wandb: Agent Starting Run: lqe8qb21 with config:
wandb: 	C: 8.830693397109062
wandb: 	penalty: l2
wandb: 	random_state: 42
wandb: 	solver: lbfgs


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.44843
f1 macro,0.49955


0.01648682815770653

In [16]:
#Don't forget to name the sweep instance   
name = 'xgb_ft_yahoo' #CHANGE HERE
sweep_config['name'] = name
#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="hyperopt")

def train_fasttext(config = None,
          train=embedded_train_yahoo, #CHANGE HERE
          val=embedded_val_yahoo): #CHANGE HERE
    # Initialize a new wandb run
    with wandb.init(config=config, group=name):
        config = wandb.config
        clf = XGBClassifier(gamma=config.gamma,
                                 n_estimators=config.n_estimators,
                                 learning_rate=config.learning_rate,
                                 max_depth=config.max_depth,
                                 random_state=config.random_state) #set the hyperparams here
        pipe = Pipeline([('clf',clf)])
        pipe.fit(train.fillna(0).drop(['label'],axis=1),train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val.fillna(0).drop(['label'],axis=1))
        pred_prob_val = pipe.predict_proba(val.fillna(0).drop(['label'],axis=1))[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
        if train['label'].nunique() <=2:
            aucpc =  average_precision_score(val['label'],pred_prob_val)
            auc = roc_auc_score(val['label'],pred_prob_val)
            #Log predictions on WandB
        else:
            aucpc = '-'
            auc = '-'
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })


#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_fasttext, count=OPT_ITER)
tracker.stop()

[codecarbon WARNING @ 15:14:08] No CPU tracking mode found. Falling back on CPU constant mode.


Create sweep with ID: 8q83yjoe
Sweep URL: https://wandb.ai/benchmark-nlp/hyperopt/sweeps/8q83yjoe


[codecarbon WARNING @ 15:14:11] We saw that you have a Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz but we don't know it. Please contact us.
wandb: Agent Starting Run: lzriiexe with config:
wandb: 	C: 3.426147217376755
wandb: 	penalty: l2
wandb: 	random_state: 42
wandb: 	solver: lbfgs


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.42979
f1 macro,0.47649


wandb: Agent Starting Run: r2tymvzh with config:
wandb: 	C: 5.357984774600081
wandb: 	penalty: l2
wandb: 	random_state: 42
wandb: 	solver: lbfgs


...

wandb: Agent Starting Run: pjlw5f0k with config:
wandb: 	C: 2.6710627849389743
wandb: 	penalty: l2
wandb: 	random_state: 42
wandb: 	solver: lbfgs


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.43143
f1 macro,0.47794


wandb: Agent Starting Run: 7lwpcbgk with config:
wandb: 	C: 8.537706451301336
wandb: 	penalty: l2
wandb: 	random_state: 42
wandb: 	solver: lbfgs


accuracy,▁
f1 macro,▁
AUC,-
AUC-PC,-
accuracy,0.43017
f1 macro,0.47666


0.006280692263080593